# Importação das Bibliotecas

In [2]:
import pandas as pd # Importação do Pandas
import numpy as np # Importação do Numpy
import boto3 # Importação do boto3
import os # Interação com o SO
from dotenv import load_dotenv # Importação do .env
from io import StringIO # Importação do StringIO


# Leitura dos Dados
Diretamente do bucket

In [3]:
load_dotenv() # Reconhece o arquivo .env

# Conexão do S3 com as Access Keys
s3 = boto3.client('s3',
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    aws_session_token=os.getenv('AWS_SESSION_TOKEN'),
    region_name='us-east-1'
)

bucket_name = 'sprint4bucket' # Identifica o bucket
object_name = 'consultas_ambulatoriais_original.csv' # Identifica o arquivo

response = s3.get_object(Bucket=bucket_name, Key=object_name) # Busca os objetos na AWS
content = response['Body'].read().decode('ISO-8859-1') # Mostra como deve ler o conteúdo

df = pd.read_csv(StringIO(content), sep=';') # StringIO possibilita a leitura do arquivo, separando as colunas por ';'

df

,data_consulta,especialidade,município,idade,sexo
0,02/01/2025 07:00,CARDIOLOGIA PEDIÁTRICA,SERRA CAIADA,3.0,F
1,02/01/2025 07:00,CARDIOLOGIA PEDIÁTRICA,AREZ,12.0,M
2,02/01/2025 07:00,CARDIOLOGIA PEDIÁTRICA,PARNAMIRIM,0.0,F
3,02/01/2025 07:00,CARDIOLOGIA PEDIÁTRICA,PARNAMIRIM,2.0,M
4,02/01/2025 07:00,CARDIOLOGIA PEDIÁTRICA,PARNAMIRIM,0.0,M
...,...,...,...,...,...
53660,31/03/2025 16:00,UROLOGIA,CEARÁ-MIRIM,56.0,M
53661,31/03/2025 16:00,UROLOGIA,NATAL,59.0,M
53662,31/03/2025 16:00,UROLOGIA,PARNAMIRIM,57.0,M
53663,31/03/2025 16:00,UROLOGIA,MARCELINO VIEIRA,51.0,M


# Solução dos Questionamentos:

## Questionamento 1:

### 1 - Separando por faixa etária, quantas pessoas fizeram cirurgias durante o mês de janeiro?

Primeiro realizo a separação da faixa etária:

In [4]:
df['idade'] = df['idade'].fillna(0).astype(int) # Convertendo coluna idade em int
df

,data_consulta,especialidade,município,idade,sexo
0,02/01/2025 07:00,CARDIOLOGIA PEDIÁTRICA,SERRA CAIADA,3,F
1,02/01/2025 07:00,CARDIOLOGIA PEDIÁTRICA,AREZ,12,M
2,02/01/2025 07:00,CARDIOLOGIA PEDIÁTRICA,PARNAMIRIM,0,F
3,02/01/2025 07:00,CARDIOLOGIA PEDIÁTRICA,PARNAMIRIM,2,M
4,02/01/2025 07:00,CARDIOLOGIA PEDIÁTRICA,PARNAMIRIM,0,M
...,...,...,...,...,...
53660,31/03/2025 16:00,UROLOGIA,CEARÁ-MIRIM,56,M
53661,31/03/2025 16:00,UROLOGIA,NATAL,59,M
53662,31/03/2025 16:00,UROLOGIA,PARNAMIRIM,57,M
53663,31/03/2025 16:00,UROLOGIA,MARCELINO VIEIRA,51,M


In [5]:
# Função condicional para criar a coluna faixa etária
df['faixa etária'] = np.where(df['idade'] <= 12, 'Criança',
np.where(df['idade'] <= 17, 'Adolescente',
np.where(df['idade'] <= 59, 'Adulto', 'Idoso')))

df

,data_consulta,especialidade,município,idade,sexo,faixa etária
0,02/01/2025 07:00,CARDIOLOGIA PEDIÁTRICA,SERRA CAIADA,3,F,Criança
1,02/01/2025 07:00,CARDIOLOGIA PEDIÁTRICA,AREZ,12,M,Criança
2,02/01/2025 07:00,CARDIOLOGIA PEDIÁTRICA,PARNAMIRIM,0,F,Criança
3,02/01/2025 07:00,CARDIOLOGIA PEDIÁTRICA,PARNAMIRIM,2,M,Criança
4,02/01/2025 07:00,CARDIOLOGIA PEDIÁTRICA,PARNAMIRIM,0,M,Criança
...,...,...,...,...,...,...
53660,31/03/2025 16:00,UROLOGIA,CEARÁ-MIRIM,56,M,Adulto
53661,31/03/2025 16:00,UROLOGIA,NATAL,59,M,Adulto
53662,31/03/2025 16:00,UROLOGIA,PARNAMIRIM,57,M,Adulto
53663,31/03/2025 16:00,UROLOGIA,MARCELINO VIEIRA,51,M,Adulto


Depois separo as colunas de horário e data

In [6]:
# Separando a data e a hora em colunas com a função de string
df[['data_consulta', 'hora_consulta']] = df['data_consulta'].str.split(' ', expand=True) # A separação ocorre no espaço 
df

,data_consulta,especialidade,município,idade,sexo,faixa etária,hora_consulta
0,02/01/2025,CARDIOLOGIA PEDIÁTRICA,SERRA CAIADA,3,F,Criança,07:00
1,02/01/2025,CARDIOLOGIA PEDIÁTRICA,AREZ,12,M,Criança,07:00
2,02/01/2025,CARDIOLOGIA PEDIÁTRICA,PARNAMIRIM,0,F,Criança,07:00
3,02/01/2025,CARDIOLOGIA PEDIÁTRICA,PARNAMIRIM,2,M,Criança,07:00
4,02/01/2025,CARDIOLOGIA PEDIÁTRICA,PARNAMIRIM,0,M,Criança,07:00
...,...,...,...,...,...,...,...
53660,31/03/2025,UROLOGIA,CEARÁ-MIRIM,56,M,Adulto,16:00
53661,31/03/2025,UROLOGIA,NATAL,59,M,Adulto,16:00
53662,31/03/2025,UROLOGIA,PARNAMIRIM,57,M,Adulto,16:00
53663,31/03/2025,UROLOGIA,MARCELINO VIEIRA,51,M,Adulto,16:00


In [7]:
# Aplicando a função de conversão para data
df['data_consulta'] = pd.to_datetime(df['data_consulta'], format='%d/%m/%Y')
df 

,data_consulta,especialidade,município,idade,sexo,faixa etária,hora_consulta
0,2025-01-02,CARDIOLOGIA PEDIÁTRICA,SERRA CAIADA,3,F,Criança,07:00
1,2025-01-02,CARDIOLOGIA PEDIÁTRICA,AREZ,12,M,Criança,07:00
2,2025-01-02,CARDIOLOGIA PEDIÁTRICA,PARNAMIRIM,0,F,Criança,07:00
3,2025-01-02,CARDIOLOGIA PEDIÁTRICA,PARNAMIRIM,2,M,Criança,07:00
4,2025-01-02,CARDIOLOGIA PEDIÁTRICA,PARNAMIRIM,0,M,Criança,07:00
...,...,...,...,...,...,...,...
53660,2025-03-31,UROLOGIA,CEARÁ-MIRIM,56,M,Adulto,16:00
53661,2025-03-31,UROLOGIA,NATAL,59,M,Adulto,16:00
53662,2025-03-31,UROLOGIA,PARNAMIRIM,57,M,Adulto,16:00
53663,2025-03-31,UROLOGIA,MARCELINO VIEIRA,51,M,Adulto,16:00


In [8]:
# Filtrando o mes de janeiro com a função de data
consultas_janeiro = df[df['data_consulta'].dt.month == 1]
consultas_janeiro

,data_consulta,especialidade,município,idade,sexo,faixa etária,hora_consulta
0,2025-01-02,CARDIOLOGIA PEDIÁTRICA,SERRA CAIADA,3,F,Criança,07:00
1,2025-01-02,CARDIOLOGIA PEDIÁTRICA,AREZ,12,M,Criança,07:00
2,2025-01-02,CARDIOLOGIA PEDIÁTRICA,PARNAMIRIM,0,F,Criança,07:00
3,2025-01-02,CARDIOLOGIA PEDIÁTRICA,PARNAMIRIM,2,M,Criança,07:00
4,2025-01-02,CARDIOLOGIA PEDIÁTRICA,PARNAMIRIM,0,M,Criança,07:00
...,...,...,...,...,...,...,...
15954,2025-01-31,UROLOGIA,TAIPU,82,M,Idoso,16:00
15955,2025-01-31,UROLOGIA,NaN,30,NaN,Adulto,16:00
15956,2025-01-31,UROLOGIA,NaN,24,F,Adulto,16:00
15957,2025-01-31,ENFERMAGEM CLINICA,ALEXANDRIA,58,F,Adulto,17:08


In [9]:
# Utilizando função de string para capitalizar os textos da coluna especialidade
df['especialidade'] = df['especialidade'].str.title()
df

,data_consulta,especialidade,município,idade,sexo,faixa etária,hora_consulta
0,2025-01-02,Cardiologia Pediátrica,SERRA CAIADA,3,F,Criança,07:00
1,2025-01-02,Cardiologia Pediátrica,AREZ,12,M,Criança,07:00
2,2025-01-02,Cardiologia Pediátrica,PARNAMIRIM,0,F,Criança,07:00
3,2025-01-02,Cardiologia Pediátrica,PARNAMIRIM,2,M,Criança,07:00
4,2025-01-02,Cardiologia Pediátrica,PARNAMIRIM,0,M,Criança,07:00
...,...,...,...,...,...,...,...
53660,2025-03-31,Urologia,CEARÁ-MIRIM,56,M,Adulto,16:00
53661,2025-03-31,Urologia,NATAL,59,M,Adulto,16:00
53662,2025-03-31,Urologia,PARNAMIRIM,57,M,Adulto,16:00
53663,2025-03-31,Urologia,MARCELINO VIEIRA,51,M,Adulto,16:00


In [10]:
# Realizando a filtragem dos itens que possuem cirurgia em janeiro (filtro & e ==)
cirurgia_idade = df[df['especialidade'].str.contains('Cirurgia') & df['faixa etária'] & df['data_consulta'].dt.month == 1]
cirurgia_idade

,data_consulta,especialidade,município,idade,sexo,faixa etária,hora_consulta
7,2025-01-02,Cirurgia Bariatrica,JOÃO CÂMARA,52,F,Adulto,07:00
8,2025-01-02,Cirurgia Bariatrica,JOÃO CÂMARA,58,F,Adulto,07:00
131,2025-01-02,Cirurgia Bariatrica,NATAL,29,F,Adulto,08:00
132,2025-01-02,Cirurgia Bariatrica,NATAL,50,F,Adulto,08:00
133,2025-01-02,Cirurgia Geral,NATAL,59,F,Adulto,08:00
...,...,...,...,...,...,...,...
53590,2025-03-31,Cirurgia Geral,PARNAMIRIM,31,F,Adulto,15:00
53591,2025-03-31,Cirurgia Geral,ANGICOS,45,F,Adulto,15:00
53592,2025-03-31,Cirurgia Geral,GOIANINHA,57,F,Adulto,15:00
53593,2025-03-31,Cirurgia Geral,ANGICOS,52,F,Adulto,15:00


In [11]:
# Demonstrando o resultado da filtragem com a função de agregação para contagem
resultado = cirurgia_idade.value_counts('faixa etária').sort_values(ascending=False)
resultado

faixa etária
Adulto         1584
Idoso           911
Criança         244
Adolescente      54
Name: count, dtype: int64

Finalizado o primeiro questionamento.

## Questionamento 2:

### 2 - Quais especialidades que registram o maior número de atendimentos para cada sexo? Separe 10 para cada.

Primeiro faço uma mudança na tabela de sexo para melhor visualização

In [12]:
# Função de string para que apareça o sexo
df['sexo'] = df['sexo'].str.replace('M', 'Masculino')
df['sexo'] = df['sexo'].str.replace('F', 'Feminino')
df['sexo'] = df['sexo'].fillna('Não informado')
df

,data_consulta,especialidade,município,idade,sexo,faixa etária,hora_consulta
0,2025-01-02,Cardiologia Pediátrica,SERRA CAIADA,3,Feminino,Criança,07:00
1,2025-01-02,Cardiologia Pediátrica,AREZ,12,Masculino,Criança,07:00
2,2025-01-02,Cardiologia Pediátrica,PARNAMIRIM,0,Feminino,Criança,07:00
3,2025-01-02,Cardiologia Pediátrica,PARNAMIRIM,2,Masculino,Criança,07:00
4,2025-01-02,Cardiologia Pediátrica,PARNAMIRIM,0,Masculino,Criança,07:00
...,...,...,...,...,...,...,...
53660,2025-03-31,Urologia,CEARÁ-MIRIM,56,Masculino,Adulto,16:00
53661,2025-03-31,Urologia,NATAL,59,Masculino,Adulto,16:00
53662,2025-03-31,Urologia,PARNAMIRIM,57,Masculino,Adulto,16:00
53663,2025-03-31,Urologia,MARCELINO VIEIRA,51,Masculino,Adulto,16:00


In [13]:
# Usando filtro para conferir se os dados realmente foram convertidos
df[df['sexo'] == 'Não informado']


,data_consulta,especialidade,município,idade,sexo,faixa etária,hora_consulta
95,2025-01-02,Otorrinolaringologia Pediatrica,NaN,7,Não informado,Criança,07:00
97,2025-01-02,Proctologia,NaN,38,Não informado,Adulto,07:00
306,2025-01-02,Cirurgia Plástica,NaN,28,Não informado,Adulto,13:00
375,2025-01-02,Cirurgia Geral,NaN,82,Não informado,Idoso,14:00
382,2025-01-02,Cirurgia Plástica,NaN,73,Não informado,Idoso,14:00
...,...,...,...,...,...,...,...
53507,2025-03-31,Neurologia,NaN,59,Não informado,Adulto,14:00
53509,2025-03-31,Neurologia,NaN,63,Não informado,Idoso,14:00
53558,2025-03-31,Urologia,NaN,67,Não informado,Idoso,14:00
53561,2025-03-31,Urologia,NaN,33,Não informado,Adulto,14:00


In [14]:
# Agrupando por especialidade e sexo
distribuicao_atend = df.groupby(['especialidade', 'sexo']).size().reset_index(name='quantidade')
distribuicao_atend.sort_values(by='quantidade', ascending=False)

,especialidade,sexo,quantidade
121,Oftalmologia,Masculino,2654
119,Oftalmologia,Feminino,2583
184,Urologia,Masculino,1905
55,Fisioterapia,Feminino,1780
53,Enfermagem Clinica,Feminino,1743
...,...,...,...
65,Gastroenterologia Pediátrica,I,1
143,Pneumologia,I,1
124,Oncologia Clínica,I,1
164,Psiquiatria,I,1


In [15]:
# Usando filtro para identificar os dados que retornam 'I'
df[df['sexo'] == 'I']

,data_consulta,especialidade,município,idade,sexo,faixa etária,hora_consulta
27588,2025-02-18,Pneumologia,NATAL,60,I,Idoso,13:00
32098,2025-02-25,Psiquiatria,NaN,0,I,Criança,09:00
33855,2025-02-27,Proctologia,NATAL,52,I,Adulto,07:00
36835,2025-03-10,Oncologia Clínica,NaN,0,I,Criança,07:00
37490,2025-03-10,Nefrologia,NaN,0,I,Criança,13:00
38622,2025-03-11,Gastroenterologia Pediátrica,NATAL,0,I,Criança,13:00
41831,2025-03-14,Oftalmologia,TAIPU,41,I,Adulto,14:00
50061,2025-03-26,Oftalmologia,TAIPU,41,I,Adulto,08:00


In [16]:
# Função de String para converter os dados com 'I' em não informado
df['sexo'] = df['sexo'].str.replace('I', 'Não informado')

In [18]:
# Repetindo a consulta 
distribuicao_atend = df.groupby(['especialidade', 'sexo']).size().reset_index(name='quantidade')
distribuicao_atend.sort_values(by='quantidade', ascending=False)

,especialidade,sexo,quantidade
118,Oftalmologia,Masculino,2654
117,Oftalmologia,Feminino,2583
177,Urologia,Masculino,1905
55,Fisioterapia,Feminino,1780
53,Enfermagem Clinica,Feminino,1743
...,...,...,...
27,Cirurgia Oncologica,Não informado,2
163,Radiologia E Diagnóstico Por Imagem,Feminino,2
91,Nefrologia Irc Doenca Min E Ossea,Masculino,2
5,Assistente Social,Feminino,1


In [19]:
# Função de agregação para mostrar a distribuição masculina
distrib_masc = df[df['sexo'] == 'Masculino'].value_counts(['especialidade', 'sexo']).reset_index(name='quantidade').head(10)
distrib_masc


,especialidade,sexo,quantidade
0,Oftalmologia,Masculino,2654
1,Urologia,Masculino,1905
2,Cardiologia,Masculino,1575
3,Oncologia Clínica,Masculino,1196
4,Neurologia,Masculino,1054
5,Nefrologia,Masculino,1000
6,Enfermagem Clinica,Masculino,936
7,Dermatologia,Masculino,913
8,Psiquiatria,Masculino,750
9,Fisioterapia,Masculino,728


In [ ]:
# Função de agregação para mostrar a distribuição feminina
distrib_femi = df[df['sexo'] == 'Feminino'].value_counts(['especialidade', 'sexo']).reset_index(name='quantidade').head(10)
distrib_femi

,especialidade,sexo,quantidade
0,Oftalmologia,Feminino,2583
1,Fisioterapia,Feminino,1780
2,Enfermagem Clinica,Feminino,1743
3,Cardiologia,Feminino,1596
4,Neurologia,Feminino,1578
5,Oncologia Clínica,Feminino,1518
6,Endocrinologia,Feminino,1492
7,Dermatologia,Feminino,1463
8,Psiquiatria,Feminino,1274
9,Otorrinolaringologia,Feminino,1008


Finalizado o segundo questionamento.

## Questionamento 3:

### 3 - Mostre as 10 cidades que possuem mais pacientes que fizeram atendimento psiquiátrico no hospital.

Primeiro realizo a capitalização dos valores da coluna município para melhor visualização.

In [21]:
# Utilizando função de string para capitalizar os textos da coluna município
df['município'] = df['município'].str.title()
df

,data_consulta,especialidade,município,idade,sexo,faixa etária,hora_consulta
0,2025-01-02,Cardiologia Pediátrica,Serra Caiada,3,Feminino,Criança,07:00
1,2025-01-02,Cardiologia Pediátrica,Arez,12,Masculino,Criança,07:00
2,2025-01-02,Cardiologia Pediátrica,Parnamirim,0,Feminino,Criança,07:00
3,2025-01-02,Cardiologia Pediátrica,Parnamirim,2,Masculino,Criança,07:00
4,2025-01-02,Cardiologia Pediátrica,Parnamirim,0,Masculino,Criança,07:00
...,...,...,...,...,...,...,...
53660,2025-03-31,Urologia,Ceará-Mirim,56,Masculino,Adulto,16:00
53661,2025-03-31,Urologia,Natal,59,Masculino,Adulto,16:00
53662,2025-03-31,Urologia,Parnamirim,57,Masculino,Adulto,16:00
53663,2025-03-31,Urologia,Marcelino Vieira,51,Masculino,Adulto,16:00


In [22]:
# Função para completar os valores da tabela
df['município'] = df['município'].fillna('Não informado')

In [23]:
# Verificação da mudança 
df[df['município'] == 'Não informado']

,data_consulta,especialidade,município,idade,sexo,faixa etária,hora_consulta
27,2025-01-02,Fisioterapia,Não informado,24,Feminino,Adulto,07:00
29,2025-01-02,Fisioterapia,Não informado,43,Feminino,Adulto,07:00
31,2025-01-02,Nefrologia Pediátrica,Não informado,10,Feminino,Criança,07:00
35,2025-01-02,Neurologia,Não informado,45,Feminino,Adulto,07:00
76,2025-01-02,Oftalmologia,Não informado,77,Feminino,Idoso,07:00
...,...,...,...,...,...,...,...
53634,2025-03-31,Urologia,Não informado,61,Masculino,Idoso,15:00
53636,2025-03-31,Urologia,Não informado,72,Não informado,Idoso,15:00
53652,2025-03-31,Fisioterapia,Não informado,40,Feminino,Adulto,16:00
53657,2025-03-31,Otorrinolaringologia,Não informado,60,Feminino,Idoso,16:00


In [24]:
# Função de agregação para filtragem dos dados
pacientes_psiq = df[df['especialidade'] == 'Psiquiatria'].value_counts('município').reset_index(name='quantidade')
pacientes_psiq.sort_values(by='quantidade', ascending=False).head(10)

,município,quantidade
0,Natal,1461
1,Parnamirim,136
2,Não informado,77
3,São Gonçalo Do Amarante,31
4,Ceará-Mirim,28
5,Extremoz,25
6,Serra De São Bento,14
7,Santana Do Matos,14
8,Rio Do Fogo,10
9,Macaíba,9


Finalizando o último questionamento.

In [ ]:
df

,data_consulta,especialidade,município,idade,sexo,faixa etária,hora_consulta
0,2025-01-02,Cardiologia Pediátrica,Serra Caiada,3,Feminino,Criança,07:00
1,2025-01-02,Cardiologia Pediátrica,Arez,12,Masculino,Criança,07:00
2,2025-01-02,Cardiologia Pediátrica,Parnamirim,0,Feminino,Criança,07:00
3,2025-01-02,Cardiologia Pediátrica,Parnamirim,2,Masculino,Criança,07:00
4,2025-01-02,Cardiologia Pediátrica,Parnamirim,0,Masculino,Criança,07:00
...,...,...,...,...,...,...,...
53660,2025-03-31,Urologia,Ceará-Mirim,56,Masculino,Adulto,16:00
53661,2025-03-31,Urologia,Natal,59,Masculino,Adulto,16:00
53662,2025-03-31,Urologia,Parnamirim,57,Masculino,Adulto,16:00
53663,2025-03-31,Urologia,Marcelino Vieira,51,Masculino,Adulto,16:00


# Etapa 3

### Salvando arquivos e subindo para o bucket

In [ ]:
df.to_csv('Consultas_Ambulatoriais_tratado.csv', index=False, encoding='utf-8') # Transformando o arquivo em CSV

In [ ]:
file_name = 'Consultas_Ambulatoriais_tratado.csv' # Nome do arquivo tratado
object_name = 'consultas_ambulatoriais_tratado.csv' # Nome do arquivo no bucket
s3.upload_file(file_name, bucket_name, object_name) # Upload do Arquivo

In [ ]:
response = s3.get_object(Bucket=bucket_name, Key=object_name) # Busca o CSv tratado na AWS
content = response['Body'].read().decode('utf-8') # Mostra como deve ler o conteúdo
df = pd.read_csv(StringIO(content)) # Leitura do arquivo

df

,data_consulta,especialidade,município,idade,sexo,faixa etária,hora_consulta
0,2025-01-02,Cardiologia Pediátrica,Serra Caiada,3,Feminino,Criança,07:00
1,2025-01-02,Cardiologia Pediátrica,Arez,12,Masculino,Criança,07:00
2,2025-01-02,Cardiologia Pediátrica,Parnamirim,0,Feminino,Criança,07:00
3,2025-01-02,Cardiologia Pediátrica,Parnamirim,2,Masculino,Criança,07:00
4,2025-01-02,Cardiologia Pediátrica,Parnamirim,0,Masculino,Criança,07:00
...,...,...,...,...,...,...,...
53660,2025-03-31,Urologia,Ceará-Mirim,56,Masculino,Adulto,16:00
53661,2025-03-31,Urologia,Natal,59,Masculino,Adulto,16:00
53662,2025-03-31,Urologia,Parnamirim,57,Masculino,Adulto,16:00
53663,2025-03-31,Urologia,Marcelino Vieira,51,Masculino,Adulto,16:00
